This is a simple utility notebook to update the games.csv file with the latest data from NBA.com. You want to run this notebook before backfilling the data to Hopsworks.ai for the first time.

In [1]:
import os

import pandas as pd

from datetime import datetime, timedelta
from pytz import timezone

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
)

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Determine How current is the existing data**

In [2]:
games_old = pd.read_csv(DATAPATH / "games.csv")

# Find the last date and season in the current dataset
last_date = games_old["GAME_DATE_EST"].max()
last_season = games_old["SEASON"].max()

# remove the time from the date
last_date = last_date.split(" ")[0]

# Determine the date of the next day to begin scraping from
start_date = datetime.strptime(last_date, "%Y-%m-%d") + timedelta(days=1)

# determine what season we are in currently
today = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
if today.month >= 10:
    current_season = today.year
else:
    current_season = today.year - 1

# determine which seasons we need to scrape to catch up the data
seasons = list(range(last_season, current_season+1))


print("Last date in dataset: ", last_date)
print("Last season in dataset: ", last_season)
print("Current season: ", current_season)
print("Seasons to scrape: ", seasons)
print("Start date: ", start_date)

# if the last date in the dataset is today, then we don't need to scrape any new data
if start_date > today:
    print("No new data to scrape")
    exit()

Last date in dataset:  2022-03-12
Last season in dataset:  2021
Current season:  2022
Seasons to scrape:  [2021, 2022]
Start date:  2022-03-13 00:00:00


**Activate Webdriver**

In [3]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('chromium')

**Scrape New Completed Games and Format Them**

In [4]:
def update_games(driver, season, start_date, end_date)-> pd.DataFrame:

    season_types = ["Regular+Season", "PlayIn", "Playoffs"]
      
    all_season_types = pd.DataFrame()

    for season_type in season_types:
        
        df = scrape_to_dataframe(api_key="", driver=driver, Season=season, DateFrom=start_date, DateTo=end_date, season_type=season_type)

        if not(df.empty):
            df = convert_columns(df)
            df = combine_home_visitor(df)
            all_season_types = pd.concat([all_season_types, df], axis=0)


    return all_season_types
    

In [5]:
new_games = pd.DataFrame()
df_season = pd.DataFrame()

for season in seasons:
    end_date = datetime.strptime(f"{season+1}-08-01", "%Y-%m-%d") # use August 1st to get all games from the current season
    print(f"Scraping season {season} from {start_date} to {end_date}")
    df_season = update_games(driver, str(season), str(start_date), str(end_date))
    new_games = pd.concat([new_games, df_season], axis=0)
    start_date = datetime.strptime(f"{season+1}-10-01", "%Y-%m-%d") # if more than 1 season, reset start date to beginning of next season


new_games

Scraping season 2021 from 2022-03-13 00:00:00 to 2022-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&Season=2021&DateFrom=2022-03-13 00:00:00&DateTo=2022-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=PlayIn&Season=2021&DateFrom=2022-03-13 00:00:00&DateTo=2022-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Playoffs&Season=2021&DateFrom=2022-03-13 00:00:00&DateTo=2022-08-01 00:00:00
Scraping season 2022 from 2022-10-01 00:00:00 to 2023-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&Season=2022&DateFrom=2022-10-01 00:00:00&DateTo=2023-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=PlayIn&Season=2022&DateFrom=2022-10-01 00:00:00&DateTo=2023-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Playoffs&Season=2022&DateFrom=2022-10-01 00:00:00&DateTo=2023-08-01 00:00:00


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-04-10,1,124,53.0,47.6,78.8,48,22,1610612741,22101224,120,50.5,35.5,77.3,32,30,1610612750,2021
1,2022-04-10,1,130,51.7,55.3,79.2,50,29,1610612737,22101221,114,46.1,37.0,75.0,34,24,1610612745,2021
2,2022-04-10,0,108,44.7,34.8,61.5,43,25,1610612764,22101217,124,53.9,54.5,71.4,48,34,1610612766,2021
3,2022-04-10,0,94,39.1,28.6,90.9,43,27,1610612761,22101226,105,46.0,34.9,83.3,48,32,1610612752,2021
4,2022-04-10,0,111,42.6,38.0,75.0,47,20,1610612748,22101227,125,49.5,39.7,85.7,48,33,1610612753,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,2023-04-16,1,115,44.1,32.3,79.3,49,23,1610612746,42200171,110,47.6,31.6,78.8,42,27,1610612756,2022
23,2023-04-15,1,101,42.0,27.6,86.4,51,18,1610612752,42200131,97,43.4,32.3,71.4,38,20,1610612739,2022
24,2023-04-15,0,99,38.8,17.2,81.8,45,23,1610612737,42200111,112,47.7,39.4,83.3,58,25,1610612738,2022
25,2023-04-15,0,101,55.7,44.8,66.7,35,23,1610612751,42200121,121,47.2,48.8,100.0,38,32,1610612755,2022


**Close Webdriver**

In [6]:
driver.close() 

**Append to Games.csv**

In [7]:
games = pd.concat([games_old, new_games], axis=0)

games.to_csv(DATAPATH / "games.csv", index=False)

games


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,2023-04-16 00:00:00,42200171,NaN,1610612746,1610612756,2022,NaN,115.0,44.100,79.300,...,23.0,49.0,NaN,110.0,47.600,78.800,31.600,27.0,42.0,1
23,2023-04-15 00:00:00,42200131,NaN,1610612752,1610612739,2022,NaN,101.0,42.000,86.400,...,18.0,51.0,NaN,97.0,43.400,71.400,32.300,20.0,38.0,1
24,2023-04-15 00:00:00,42200111,NaN,1610612737,1610612738,2022,NaN,99.0,38.800,81.800,...,23.0,45.0,NaN,112.0,47.700,83.300,39.400,25.0,58.0,0
25,2023-04-15 00:00:00,42200121,NaN,1610612751,1610612755,2022,NaN,101.0,55.700,66.700,...,23.0,35.0,NaN,121.0,47.200,100.000,48.800,32.0,38.0,0
